In [1]:
from openenergyid.pandera_poc import InputModel

# Input Model Specification

InputModel.to_json_schema()

{'title': 'InputModel',
 'type': 'object',
 'properties': {'index': {'type': 'array', 'items': {'type': 'integer'}},
  'column1': {'type': 'array', 'items': {'type': 'integer'}},
  'column2': {'type': 'array', 'items': {'type': 'number'}},
  'column3': {'type': 'array', 'items': {'type': 'string'}}}}

In [2]:
# Input data is pure python
import pandas as pd

df = pd.DataFrame(
    {
        "column1": [1, 4, 0, 10, 9],
        "column2": [-1.3, -1.4, -2.9, -10.1, -20.4],
        "column3": ["value_1", "value_2", "value_3", "value_2", "value_1"],
    },
    index=range(5),
)

df

,column1,column2,column3
0,1,-1.3,value_1
1,4,-1.4,value_2
2,0,-2.9,value_3
3,10,-10.1,value_2
4,9,-20.4,value_1


In [3]:
from openenergyid.pandera_poc import analyse, InputModel, OutputModel

InputModel.validate(df)
result = analyse(df)
OutputModel.validate(result)

result

,column1,column2,column3
0,1,-1.3,value_1
1,4,-1.4,value_2
2,0,-2.9,value_3
3,10,-10.1,value_2
4,9,-20.4,value_1
